# Forest Identification

### Import Libraries

In [1]:
import os
import numpy as np
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

2024-05-28 14:19:55.931859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 14:19:56.044844: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-28 14:19:56.581785: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-05-28 14:19:56.581849

## Data Preprocessing

### Function to extract a zip file

In [2]:
def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted {zip_path} to {extract_to}")

### Function to load and preprocess images

In [3]:
def load_and_preprocess_images(image_folder, metadata, has_mask=False):
    images = []
    labels = []
    for _, row in metadata.iterrows():
        img_path = os.path.join(image_folder, row['sat_image_path'])
        if os.path.exists(img_path):
            img = Image.open(img_path)
            img = img.resize((128, 128))
            img = np.array(img) / 255.0  # Normalize pixel values
            images.append(img)
            if has_mask:
                mask_path = os.path.join(image_folder, row['mask_path'])
                if os.path.exists(mask_path):
                    label = mask_to_label(mask_path)
                    labels.append(label)
    images = np.array(images)
    if has_mask:
        labels = np.array(labels)
        return images, labels
    return images

### Function to convert mask image to label

In [4]:
def mask_to_label(mask_path):
    mask = Image.open(mask_path).convert('RGB')
    mask = mask.resize((128, 128), Image.NEAREST)
    mask = np.array(mask)
    label = np.zeros((128, 128), dtype=np.int32)
    for rgb, idx in class_mapping.items():
        label[np.all(mask == rgb, axis=-1)] = idx
    return label


### Define paths

In [9]:
dataset_zip_path = 'dataset.zip'
dataset_path = './dataset'
# Define paths to image folders
train_folder = 'train'
val_folder = 'valid'
test_folder = 'test'
train_mask_folder = 'train'  # Only training has masks

### Step 1: Extract the dataset (do this only once)

In [10]:
#extract_zip(dataset_zip_path, dataset_path)

### Step 2: Load the class dictionary and metadata

In [11]:
# Load the class dictionary
class_dict = pd.read_csv(dataset_path + '/class_dict.csv')

# Create a dictionary to map RGB values to class indices
class_mapping = {}
for i, row in class_dict.iterrows():
    rgb = (row['r'], row['g'], row['b'])
    class_mapping[rgb] = i

# Load metadata
metadata = pd.read_csv(dataset_path + '/metadata.csv')

### Step 3: Load and preprocess the images for each split

In [12]:
X_train, y_train = load_and_preprocess_images(dataset_path, metadata[metadata['split'] == 'train'], True)
X_val = load_and_preprocess_images(val_folder, metadata[metadata['split'] == 'val'])
X_test = load_and_preprocess_images(test_folder, metadata[metadata['split'] == 'test'])

In [13]:
np.save('X_train.npy', X_train)
np.save('X_val.npy', X_val)
np.save('X_test.npy', X_test)
if train_mask_folder:
    np.save('y_train.npy', y_train)

print("Data preparation completed successfully.")

Data preparation completed successfully.


## Data Augmentation

## Model Building

In [14]:
# Enable eager execution for better error messages
tf.config.run_functions_eagerly(True)

def unet_model(input_size=(128, 128, 3), num_classes=7):
    inputs = Input(input_size)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model


# Instantiate and compile the model
model = unet_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Summary of the model
model.summary()

2024-05-28 14:29:30.023545: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12591 MB memory:  -> device: 0, name: NVIDIA A16, pci bus id: 0000:1b:00.0, compute capability: 8.6
2024-05-28 14:29:30.027907: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13134 MB memory:  -> device: 1, name: NVIDIA A16, pci bus id: 0000:1c:00.0, compute capability: 8.6
2024-05-28 14:29:30.029425: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 13134 MB memory:  -> device: 2, name: NVIDIA A16, pci bus id: 0000:1d:00.0, compute capability: 8.6
2024-05-28 14:29:30.030939: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 13134 MB memory:  -> device: 3, name: NVIDIA A16, pci bus id: 0000:1e:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

## Model Training

In [16]:
# Load the preprocessed data
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')

# Ensure y_train has the correct shape for sparse_categorical_crossentropy
if len(y_train.shape) == 3:
    y_train = np.expand_dims(y_train, axis=-1)  # Convert from (batch, height, width) to (batch, height, width, 1)

# Define data augmentation
data_gen_args = dict(rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.1,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     fill_mode='nearest')

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Fit the generators to the data
image_datagen.fit(X_train, augment=True)
mask_datagen.fit(y_train, augment=True)

# Define generator to yield batches of augmented data
def train_generator(image_datagen, mask_datagen, batch_size):
    image_generator = image_datagen.flow(X_train, batch_size=batch_size, seed=1)
    mask_generator = mask_datagen.flow(y_train, batch_size=batch_size, seed=1)
    while True:
        x_batch = image_generator.next()
        y_batch = mask_generator.next()
        yield x_batch, y_batch

batch_size = 8
train_gen = train_generator(image_datagen, mask_datagen, batch_size)
    
# Train the model with data augmentation
history = model.fit(train_gen, steps_per_epoch=len(X_train) // batch_size, epochs=30)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/30


2024-05-28 14:32:46.195003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8400
2024-05-28 14:32:55.149139: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


100/100 [==============================] - 47s 256ms/step - loss: 1.4256 - accuracy: 0.5571
Epoch 2/30
100/100 [==============================] - 33s 256ms/step - loss: 1.3366 - accuracy: 0.5642
Epoch 3/30
100/100 [==============================] - 25s 255ms/step - loss: 1.2873 - accuracy: 0.5598
Epoch 4/30
100/100 [==============================] - 25s 254ms/step - loss: 1.2354 - accuracy: 0.5928
Epoch 5/30
100/100 [==============================] - 25s 254ms/step - loss: 1.2181 - accuracy: 0.5894
Epoch 6/30
100/100 [==============================] - 25s 251ms/step - loss: 1.1842 - accuracy: 0.5964
Epoch 7/30
100/100 [==============================] - 25s 250ms/step - loss: 1.1555 - accuracy: 0.5979
Epoch 8/30
100/100 [==============================] - 25s 251ms/step - loss: 1.1465 - accuracy: 0.5956
Epoch 9/30
100/100 [==============================] - 25s 252ms/step - loss: 1.0772 - accuracy: 0.6124
Epoch 10/30
100/100 [==============================] - 25s 254ms/step - loss: 1.0523

In [ ]:
# Plot training accuracy and loss values
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Model Evaluation

In [ ]:
# Load the preprocessed validation data
X_val = np.load('X_val.npy')

# Predict on validation set
predictions = model.predict(X_val)

# Function to visualize predictions
def visualize_predictions(images, predictions, num_images=5):
    for i in range(num_images):
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.imshow(images[i])
        plt.title('Original Image')

        plt.subplot(1, 3, 2)
        plt.imshow(np.argmax(predictions[i], axis=-1), cmap='jet')
        plt.title('Predicted Segmentation')

        plt.show()

# Visualize predictions
visualize_predictions(X_val, predictions)

## Prediction

In [ ]:
def predict_land_cover(image_path, model):
    img = Image.open(image_path)
    img = img.resize((128, 128))  # Resize to match the input shape of the model
    img = np.array(img) / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction, axis=-1)
    return predicted_class[0]

# Predict on a new image
new_image_path = 'path_to_new_image.jpg'
result = predict_land_cover(new_image_path, model)
print(f"Predicted class map:\n{result}")